<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=807d5d6af38225cb665d56d321aa837114522dd55a28805f5b2805ecb781b2b7
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-05 09:23:28
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.40 C
-------------------
Today PnL: -2.39 K (-0.02%)
Current PnL: -18.19 L (-12.23%)
CY Booked + Current PnL: -8.33 L (-5.6%)
-------------------
Total profit:  2.97 L
Total loss:  -21.15 L
-------------------
Total Booked + Current PnL: 18.65 L (15.33%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.05%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.90 L (56.41%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.78%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DIXON,18940.15,-28.34,71.0,H-MC,7.44,143416.0,28480.0,8103.0,0.42,24.78,5.65,31.83,81.0,3.51,1.08,48.65,X40N,ATH,IT
4,APOLLOHOSP,8285.00,-13.31,63.0,H-LC,10.09,173250.0,17073.0,9026.0,-0.32,10.93,5.21,16.71,22.0,1.89,1.31,30.28,X40N,BTT,HEALTHCARE
79,TTKPRESTIG,770.00,104.91,64.0,M-SC,6.36,88268.0,-12509.0,12605.0,-0.24,-12.41,14.28,0.09,245.0,-0.99,0.67,15.96,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,50.0,M-LC,4.52,215140.0,13660.0,13554.0,0.30,6.78,6.30,13.51,60.0,1.01,1.62,20.23,XY25,NTT,BANKS
26,DMART,5201.00,-4.50,74.0,H-LC,15.56,238429.0,53948.0,16428.0,0.76,29.24,6.89,38.14,38.0,3.28,1.80,42.95,X40N,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIPRULI,790.00,-20.77,40.0,H-MC,2.18,136323.0,535.0,42219.0,0.57,0.39,30.97,31.48,107.0,0.01,1.03,12.56,X40,ATH,INSURANCE
31,HAVELLS,2069.16,-0.51,60.0,H-MC,1.92,246698.0,-1033.0,76082.0,-0.21,-0.42,30.84,30.30,92.0,-0.01,1.86,12.82,X40,ATH,ELECTRICAL
53,PGHH,17973.08,-30.02,59.0,H-MC,4.98,202635.0,1815.0,66971.0,-0.12,0.90,33.05,34.25,80.0,0.03,1.53,6.15,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,-0.65,38.0,H-LC,9.89,227273.0,-14651.0,77932.0,0.30,-6.06,34.29,26.15,8.0,-0.19,1.71,7.40,X40,ATH,IT
85,VBL,671.64,-14.47,41.0,H-LC,6.90,307835.0,-8007.0,120671.0,-1.43,-2.54,39.20,35.67,5.0,-0.07,2.32,10.97,X40N,ATH,FMCG
45,JIOFIN,387.00,-1.31,42.0,H-LC,14.00,218190.0,784.0,52715.0,0.73,0.36,24.16,24.61,48.0,0.01,1.65,55.38,XY24,BTT,FINANCE
58,RELIANCE,1533.00,-14.79,43.0,H-LC,7.24,213720.0,3354.0,25433.0,0.79,1.59,11.90,13.68,37.0,0.13,1.61,18.38,XY25,NTT,REFINERIES
41,INFY,2275.00,-19.60,44.0,H-LC,10.90,312769.0,-119.0,171804.0,0.37,-0.04,54.93,54.87,3.0,-0.00,2.36,6.61,X40,BTT,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-5.20,55.0,H-LC,1.88,235305.0,-10846.0,90051.0,0.36,-4.41,38.27,32.18,16.0,-0.12,1.77,30.05,X200,ATH,IT
52,NESTLEIND,1377.00,-8.60,60.0,H-LC,2.72,281174.0,15748.0,41051.0,-0.91,5.93,14.60,21.40,11.0,0.38,2.12,13.70,XY25,NTT,FMCG
43,ITC,452.00,-37.92,51.0,H-LC,3.01,200881.0,743.0,20148.0,-1.23,0.37,10.03,10.44,4.0,0.04,1.52,5.96,X40,NTT,FMCG
19,CIPLA,1795.00,-18.75,59.0,H-LC,5.62,216570.0,12070.0,29345.0,0.19,5.90,13.55,20.25,10.0,0.41,1.63,15.15,X40N,BTT,PHARMA
85,VBL,671.64,-14.47,41.0,H-LC,6.90,307835.0,-8007.0,120671.0,-1.43,-2.54,39.20,35.67,5.0,-0.07,2.32,10.97,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.00,3845.95,76.0,M-SC,9.80,123086.0,-24282.0,76104.0,2.45,-16.48,61.83,35.16,236.0,-0.32,0.93,25.72,XY24,NTT,AUTO
17,CAMS,5211.76,-6.05,48.0,H-SC,5.49,106876.0,4872.0,39052.0,1.39,4.78,36.54,43.06,122.0,0.12,0.81,22.71,X40N,ATH,MISC
84,VALIANTORG,838.00,-308.98,43.0,H-SC,9.56,88920.0,-44366.0,128961.0,1.18,-33.29,145.03,63.47,139.0,-0.34,0.67,49.05,XR,NTT,CHEMICALS
5,ASIANPAINT,4250.00,-9.53,65.0,H-LC,8.20,228404.0,-23364.0,145585.0,1.05,-9.28,63.74,48.55,27.0,-0.16,1.72,22.86,X40,BTT,PAINTS
86,VOLTAS,1918.49,1.30,62.0,H-MC,4.01,212490.0,20748.0,75285.0,1.00,10.82,35.43,50.08,99.0,0.28,1.60,18.35,XY25,ATH,AC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VAIBHAVGBL,521.00,39.94,46.0,H-SC,11.67,93536.0,-58455.0,132578.0,-2.12,-38.46,141.74,48.77,125.0,-0.44,0.71,8.87,XR,NTT,JEWELLERY
12,BATAINDIA,2096.00,-32.15,63.0,M-SC,9.93,101509.0,-27161.0,72457.0,-1.81,-21.11,71.38,35.20,219.0,-0.37,0.77,16.51,X40,NTT,FOOTWEAR
85,VBL,671.64,-14.47,41.0,H-LC,6.90,307835.0,-8007.0,120671.0,-1.43,-2.54,39.20,35.67,5.0,-0.07,2.32,10.97,X40N,ATH,FMCG
16,CAMPUS,393.00,-28.17,45.0,M-SC,3.14,149732.0,-18043.0,73878.0,-1.33,-10.75,49.34,33.28,210.0,-0.24,1.13,15.93,XY24,NTT,FOOTWEAR
57,RELAXO,1176.00,-32.13,59.0,H-SC,4.82,62963.0,-52575.0,87563.0,-1.28,-45.50,139.07,30.28,136.0,-0.60,0.47,23.79,X40N,NTT,FOOTWEAR


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,104.91,64.0,M-SC,6.36,88268.0,-12509.0,12605.0,-0.24,-12.41,14.28,0.09,245.0,-0.99,0.67,15.96,OX40N,NTT,DURABLES
80,UJJIVANSFB,60.0,84.36,45.0,H-SC,13.50,116397.0,-26082.0,45604.0,0.14,-18.31,39.18,13.70,163.0,-0.57,0.88,37.38,OX40N,NTT,BANKS
39,INDIGOPNTS,1408.0,115.96,51.0,M-SC,3.53,141496.0,-33063.0,33096.0,-1.02,-18.94,23.39,0.02,221.0,-1.00,1.07,23.08,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-68.34,53.0,H-SC,3.50,218025.0,-51642.0,87973.0,-0.41,-19.15,40.35,13.47,138.0,-0.59,1.64,10.27,XY24,NTT,PAINTS
18,CERA,9475.0,-20.29,44.0,H-SC,2.78,113769.0,-30034.0,56782.0,-0.70,-20.89,49.91,18.60,149.0,-0.53,0.86,25.55,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-27.08,48.0,H-LC,14.51,290996.0,-54960.0,178817.0,-0.06,-15.89,61.45,35.80,1.0,-0.31,2.19,3.09,X40,BTT,IT
41,INFY,2275.00,-19.60,44.0,H-LC,10.90,312769.0,-119.0,171804.0,0.37,-0.04,54.93,54.87,3.0,-0.00,2.36,6.61,X40,BTT,IT
43,ITC,452.00,-37.92,51.0,H-LC,3.01,200881.0,743.0,20148.0,-1.23,0.37,10.03,10.44,4.0,0.04,1.52,5.96,X40,NTT,FMCG
85,VBL,671.64,-14.47,41.0,H-LC,6.90,307835.0,-8007.0,120671.0,-1.43,-2.54,39.20,35.67,5.0,-0.07,2.32,10.97,X40N,ATH,FMCG
1,ABB,7934.00,-40.64,48.0,H-LC,13.08,248165.0,-13454.0,132669.0,0.14,-5.14,53.46,45.57,7.0,-0.10,1.87,5.60,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,0.81,78.0,H-MC,21.07,187250.0,36337.0,20972.0,-0.02,24.08,11.20,37.97,93.0,1.73,1.41,55.47,AR,ATH,AUTO
25,DIXON,18940.15,-28.34,71.0,H-MC,7.44,143416.0,28480.0,8103.0,0.42,24.78,5.65,31.83,81.0,3.51,1.08,48.65,X40N,ATH,IT
26,DMART,5201.00,-4.50,74.0,H-LC,15.56,238429.0,53948.0,16428.0,0.76,29.24,6.89,38.14,38.0,3.28,1.80,42.95,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,13.00,64.0,H-MC,3.38,195020.0,30350.0,37619.0,0.32,18.43,19.29,41.28,89.0,0.81,1.47,27.88,X40N,ATH,AC
86,VOLTAS,1918.49,1.30,62.0,H-MC,4.01,212490.0,20748.0,75285.0,1.00,10.82,35.43,50.08,99.0,0.28,1.60,18.35,XY25,ATH,AC
33,HEROMOTOCO,5949.07,0.81,78.0,H-MC,21.07,187250.0,36337.0,20972.0,-0.02,24.08,11.20,37.97,93.0,1.73,1.41,55.47,AR,ATH,AUTO
23,DABUR,735.00,-1.31,66.0,H-MC,4.46,214636.0,17608.0,72010.0,-0.43,8.94,33.55,45.49,102.0,0.24,1.62,23.14,XY24,BTT,FMCG
17,CAMS,5211.76,-6.05,48.0,H-SC,5.49,106876.0,4872.0,39052.0,1.39,4.78,36.54,43.06,122.0,0.12,0.81,22.71,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,0.81,78.0,H-MC,21.07,187250.0,36337.0,20972.0,-0.02,24.08,11.20,37.97,93.0,1.73,1.41,55.47,AR,ATH,AUTO
51,NATIONALUM,247.44,-44.49,72.0,H-MC,0.56,110500.0,9996.0,20641.0,0.88,9.95,18.68,30.49,79.0,0.48,0.83,45.57,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-40.54,57.0,M-SC,4.26,100568.0,9070.0,69684.0,-0.63,9.91,69.29,86.07,223.0,0.13,0.76,46.04,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.84,44.0,H-SC,12.21,134408.0,11072.0,122688.0,-0.98,8.98,91.28,108.45,119.0,0.09,1.01,33.34,AR,ATH,MISC
88,WIPRO,420.00,-12.89,45.0,M-LC,6.91,152687.0,1742.0,107705.0,0.53,1.15,70.54,72.51,53.0,0.02,1.15,7.25,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-3.83,46.0,H-SC,15.65,89466.0,-11497.0,106026.0,-0.89,-11.39,118.51,93.63,148.0,-0.11,0.67,33.26,SR,ATH,CHEMICALS
21,COFFEEDAY,80.00,-39.64,61.0,L-SC,36.90,88454.0,-25095.0,65146.0,-0.32,-22.10,73.65,35.27,268.0,-0.39,0.67,116.49,XR,NTT,HOTELS
50,MASFIN,398.61,-17.80,52.0,H-SC,12.88,93450.0,-4530.0,26129.0,0.27,-4.62,27.96,22.05,152.0,-0.17,0.70,35.84,XR,ATH,FINANCE
51,NATIONALUM,247.44,-44.49,72.0,H-MC,0.56,110500.0,9996.0,20641.0,0.88,9.95,18.68,30.49,79.0,0.48,0.83,45.57,MH,ATH,METALS
80,UJJIVANSFB,60.00,84.36,45.0,H-SC,13.50,116397.0,-26082.0,45604.0,0.14,-18.31,39.18,13.70,163.0,-0.57,0.88,37.38,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-39.64,61.0,L-SC,36.90,88454.0,-25095.0,65146.0,-0.32,-22.10,73.65,35.27,268.0,-0.39,0.67,116.49,XR,NTT,HOTELS
33,HEROMOTOCO,5949.07,0.81,78.0,H-MC,21.07,187250.0,36337.0,20972.0,-0.02,24.08,11.20,37.97,93.0,1.73,1.41,55.47,AR,ATH,AUTO
7,ATULAUTO,844.00,3845.95,76.0,M-SC,9.80,123086.0,-24282.0,76104.0,2.45,-16.48,61.83,35.16,236.0,-0.32,0.93,25.72,XY24,NTT,AUTO
26,DMART,5201.00,-4.50,74.0,H-LC,15.56,238429.0,53948.0,16428.0,0.76,29.24,6.89,38.14,38.0,3.28,1.80,42.95,X40N,NTT,FMCG
57,RELAXO,1176.00,-32.13,59.0,H-SC,4.82,62963.0,-52575.0,87563.0,-1.28,-45.50,139.07,30.28,136.0,-0.60,0.47,23.79,X40N,NTT,FOOTWEAR


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.91
1,25,43.90
2,50,73.97


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.72
LC    36.39
MC    25.85
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.69
X40      15.73
X40N     14.42
XY25     10.09
AR        9.49
XR        9.04
OX40N     7.44
X200      1.77
MH        1.75
X5K       1.44
SR        1.10
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.09
H-MC    22.86
H-SC    22.53
M-SC    13.61
M-LC     7.21
M-MC     2.67
L-SC     1.58
L-LC     1.09
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.99,0.54,33.61
IT,13.48,-15.37,72.57
FINANCE,7.40,-22.81,68.09
BANKS,7.40,-16.15,65.47
PAINTS,6.08,-14.47,40.09
MISC,5.85,-18.47,86.38
ELECTRICAL,5.74,-8.11,67.70
AUTO,4.82,-5.78,51.81
AC,3.80,4.07,28.41


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2794101.0,22
AR,1219744.0,10
XR,1093740.0,13
X40,881067.0,10
X40N,521988.0,11
OX40N,471843.0,10
XY25,411805.0,7
SR,197266.0,2
X5K,126075.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2785665.0,27
M-SC,1443014.0,17
H-LC,1394126.0,16
H-MC,1171349.0,17
M-LC,422545.0,5
M-MC,323214.0,2
L-SC,251427.0,3
L-MC,59510.0,1
L-LC,39295.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,934472.0,6
M-SC,XY24,852801.0,7
H-SC,AR,665477.0,5
H-LC,X40,594286.0,5
H-SC,XR,527194.0,6
H-MC,XY24,455688.0,4
H-LC,AR,372807.0,2
M-MC,XY24,323214.0,2
H-LC,X40N,217783.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
